# Algorithm 2

In [2]:
import networkx as nx
import glob, os
import numpy as np
from tqdm.notebook import tqdm
from itertools import islice
import statistics
from numpy.random import choice
import math
import heapq

In [1]:
def nodes_connected(G, u, v):
  return u in G.neighbors(v)

In [3]:
def s(id):
  if int(id[0]) <= int(id[1]): return id
  else: return id[1], id[0]

In [4]:
def sort_dict_by_value(d, reverse = False):
  return dict(sorted(d.items(), key = lambda x: x[1], reverse = reverse))

# Train oracle for oregon dataset

In [5]:
data_list = []
with open("/content/oregon/oregon_1.txt") as f:
  lines = f.readlines()[4:]
  data_list.append(lines)

with open("/content/oregon/oregon_4.txt") as f:
  lines = f.readlines()[4:]
  data_list.append(lines)


train_list = data_list[0]
test_list = data_list[1]


G_train = nx.Graph()
for el in train_list:
    values = el.strip().split()
    G_train.add_edges_from([(values[0],values[1])])


G_test = nx.Graph()
for el in test_list:
    values = el.strip().split()
    G_test.add_edges_from([(values[0],values[1])])


score_matrix = dict()
for idx,el in enumerate(list(G_train.edges())):
  u = el[0]
  v = el[1]  
  u_neigh = set(list(G_train.neighbors(u)))
  v_neigh = set(list(G_train.neighbors(v)))
  common_neighbour = set.intersection(u_neigh,v_neigh) 
  N_uv = len(common_neighbour)
  score_matrix[s((u,v))] = N_uv



score_matrix_sorted = sort_dict_by_value(score_matrix, True)

oracle = dict()
for key, value in islice(score_matrix_sorted.items(), 0, int(len(score_matrix_sorted)/10)):
  oracle.update({s(key):value})

# Train oracle for caida_2006 dataset

In [ ]:
caida_2006_data_list = []
with open("/content/caida_2006/caida_1.txt") as f:
  lines = f.readlines()[8:]
  caida_2006_data_list.append(lines)

with open("/content/caida_2006/caida_30.txt") as f:
  lines = f.readlines()[8:]
  caida_2006_data_list.append(lines)

caida_2006_train_list = caida_2006_data_list[0]
caida_2006_test_list = caida_2006_data_list[1]


caida_2006_G_train = nx.Graph()
for el in caida_2006_train_list:
    values = el.strip().split()
    caida_2006_G_train.add_edges_from([(values[0],values[1])])


caida_2006_G_test = nx.Graph()
for el in caida_2006_test_list:
    values = el.strip().split()
    caida_2006_G_test.add_edges_from([(values[0],values[1])])


caida_2006_score_matrix = dict()
for idx,el in enumerate(list(caida_2006_G_train.edges())):
  u = el[0]
  v = el[1]  
  u_neigh = set(list(caida_2006_G_train.neighbors(u)))
  v_neigh = set(list(caida_2006_G_train.neighbors(v)))
  common_neighbour = set.intersection(u_neigh,v_neigh) 
  N_uv = len(common_neighbour)
  caida_2006_score_matrix[s((u,v))] = N_uv



caida_2006_score_matrix_sorted = sort_dict_by_value(caida_2006_score_matrix, True)

caida_2006_oracle = dict()
for key, value in islice(caida_2006_score_matrix_sorted.items(), 0, int(len(caida_2006_score_matrix_sorted)/10)):
  caida_2006_oracle.update({s(key):value})

# Train oracle for caida_2007 dataset

In [ ]:
caida_2007_data_list = []
with open("/content/caida_2007/caida_1.txt") as f:
  lines = f.readlines()[8:]
  caida_2007_data_list.append(lines)

with open("/content/caida_2007/caida_25.txt") as f:
  lines = f.readlines()[8:]
  caida_2007_data_list.append(lines)

caida_2007_train_list = caida_2007_data_list[0]
caida_2007_test_list = caida_2007_data_list[1]


caida_2007_G_train = nx.Graph()
for el in caida_2007_train_list:
    values = el.strip().split()
    caida_2007_G_train.add_edges_from([(values[0],values[1])])


caida_2007_G_test = nx.Graph()
for el in caida_2007_test_list:
    values = el.strip().split()
    caida_2007_G_test.add_edges_from([(values[0],values[1])])


caida_2007_score_matrix = dict()
for idx,el in enumerate(list(caida_2007_G_train.edges())):
  u = el[0]
  v = el[1]  
  u_neigh = set(list(caida_2007_G_train.neighbors(u)))
  v_neigh = set(list(caida_2007_G_train.neighbors(v)))
  common_neighbour = set.intersection(u_neigh,v_neigh) 
  N_uv = len(common_neighbour)
  caida_2007_score_matrix[s((u,v))] = N_uv



caida_2007_score_matrix_sorted = sort_dict_by_value(caida_2007_score_matrix, True)

caida_2007_oracle = dict()
for key, value in islice(caida_2007_score_matrix_sorted.items(), 0, int(len(caida_2007_score_matrix_sorted)/10)):
  caida_2007_oracle.update({s(key):value})

# Test for oregon dataset

In [7]:
m = G_test.number_of_edges()
n = G_test.number_of_nodes()

X = set()
eps = 0.05
alpha = 1
beta = 10
c = 0.15
K = 1
T = sum(score_matrix.values())/3

H = int((eps**-2)*((np.log10(K/eps))**2)*(alpha+((m*beta)/T))*0.5)

set_of_edge_score = dict()
set_of_edges = dict()
set_of_ervs = dict()
current_maximum = dict()
C = dict()

for i in range(int(c*(eps**-2))):
  set_of_edges.update({i:set()})
  set_of_ervs.update({i:dict()})
  current_maximum.update({i:0})
  C.update({i:dict()})

pbar = tqdm(G_test.nodes())
for y in pbar:

  for i in range(int(c*(eps**-2))): #Update the counters for each sample
    for ab in set_of_edges[i]:
      if nodes_connected(G_test, str(s(ab)[0]), y) and nodes_connected(G_test, str(s(ab)[1]), y):
        C[i][s(ab)] += 1
 
  for yx in list(G_test.edges(y)):
    yx = s(yx)
    # xy = tuple(reversed(yx))
    for i in range(int(c*(eps**-2))):
      if yx in set_of_edges[i]:
        current_maximum[i] = max(current_maximum[i], C[i][yx]/set_of_ervs[i][yx])
      else:
        R_yx = oracle[s(yx)] + beta if s(yx) in oracle else beta
        set_of_ervs[i][yx] = np.random.exponential(1)
        C[i][yx] = 0
        set_of_edges[i].add(yx)
        
        temp_score = R_yx / set_of_ervs[i][yx]
        set_of_edge_score[(yx[0],yx[1],i)] = temp_score

  M =  np.inf
  sum_of_s_i = 0
  set_of_edge_score_sorted = dict()
  set_of_edge_score_sorted = sort_dict_by_value(set_of_edge_score, True)
  for el  in set_of_edge_score_sorted:
    sum_of_s_i += 1
    if sum_of_s_i > H :
      break
    else:
      M = set_of_edge_score_sorted[el]
  copy = dict()
  copy = dict(set_of_edge_score)
  set_of_edge_score = dict()

  for e in copy:
    score = copy[e]
    edge = s(tuple((e[0],e[1])))
    idx = e[2]
    if score < M:
      set_of_edges[idx].remove(s(edge))
      del set_of_ervs[idx][s(edge)]
      del C[idx][s(edge)]
    else:
      set_of_edge_score[e] = score
          


for i in range(int(c*(eps**-2))):
  X.add(current_maximum[i])

temp_count= int(np.log(2) * sorted(X)[len(X)//2])
print("Predicted number of Triangles of oregon #4: ", temp_count)
real_tri_count = sum(nx.triangles(G_test).values())/3
print("real number of Triangles of oregon #4: ", real_tri_count)

  0%|          | 0/10859 [00:00<?, ?it/s]

Predicted number of Triangles of oregon #4:  20452
real number of Triangles of oregon #4:  19108.0


# Test for caida_2006 dataset

In [ ]:
m = caida_2006_G_test.number_of_edges()
n = caida_2006_G_test.number_of_nodes()

X = set()
eps = 0.05
alpha = 1
beta = 10
c = 0.15
K = 1
T = sum(caida_2006_score_matrix.values())/3

H = int((eps**-2)*((np.log10(K/eps))**2)*(alpha+((m*beta)/T))*0.5)

set_of_edge_score = dict()
set_of_edges = dict()
set_of_ervs = dict()
current_maximum = dict()
C = dict()

for i in range(int(c*(eps**-2))):
  set_of_edges.update({i:set()})
  set_of_ervs.update({i:dict()})
  current_maximum.update({i:0})
  C.update({i:dict()})

pbar = tqdm(caida_2006_G_test.nodes())
for y in pbar:

  for i in range(int(c*(eps**-2))): #Update the counters for each sample
    for ab in set_of_edges[i]:
      if nodes_connected(caida_2006_G_test, str(s(ab)[0]), y) and nodes_connected(caida_2006_G_test, str(s(ab)[1]), y):
        C[i][s(ab)] += 1
 
  for yx in list(caida_2006_G_test.edges(y)):
    yx = s(yx)
    # xy = tuple(reversed(yx))
    for i in range(int(c*(eps**-2))):
      if yx in set_of_edges[i]:
        current_maximum[i] = max(current_maximum[i], C[i][yx]/set_of_ervs[i][yx])
      else:
        R_yx = caida_2006_oracle[s(yx)] + beta if s(yx) in caida_2006_oracle else beta
        set_of_ervs[i][yx] = np.random.exponential(1)
        C[i][yx] = 0
        set_of_edges[i].add(yx)
        
        temp_score = R_yx / set_of_ervs[i][yx]
        set_of_edge_score[(yx[0],yx[1],i)] = temp_score

  M =  np.inf
  sum_of_s_i = 0
  set_of_edge_score_sorted = dict()
  set_of_edge_score_sorted = sort_dict_by_value(set_of_edge_score, True)
  for el  in set_of_edge_score_sorted:
    sum_of_s_i += 1
    if sum_of_s_i > H :
      break
    else:
      M = set_of_edge_score_sorted[el]
  copy = dict()
  copy = dict(set_of_edge_score)
  set_of_edge_score = dict()

  for e in copy:
    score = copy[e]
    edge = s(tuple((e[0],e[1])))
    idx = e[2]
    if score < M:
      set_of_edges[idx].remove(s(edge))
      del set_of_ervs[idx][s(edge)]
      del C[idx][s(edge)]
    else:
      set_of_edge_score[e] = score
          


for i in range(int(c*(eps**-2))):
  X.add(current_maximum[i])

temp_count= int(np.log(2) * sorted(X)[len(X)//2])
print("Predicted number of Triangles of caida_2006 #30: ", temp_count)
real_tri_count = sum(nx.triangles(caida_2006_G_test).values())/3
print("Real number of Triangles of caida_2006 #30: ", real_tri_count)

  0%|          | 0/22602 [00:00<?, ?it/s]

Predicted number of Triangles of caida_2006 #30:  35423
real number of Triangles of caida_2006 #30:  34926.0


# Test for caida_2007 dataset

In [ ]:
m = caida_2007_G_test.number_of_edges()
n = caida_2007_G_test.number_of_nodes()

X = set()
eps = 0.05
alpha = 1
beta = 10
c = 0.15
K = 1
T = sum(caida_2007_score_matrix.values())/3

H = int((eps**-2)*((np.log10(K/eps))**2)*(alpha+((m*beta)/T))*0.5)

set_of_edge_score = dict()
set_of_edges = dict()
set_of_ervs = dict()
current_maximum = dict()
C = dict()

for i in range(int(c*(eps**-2))):
  set_of_edges.update({i:set()})
  set_of_ervs.update({i:dict()})
  current_maximum.update({i:0})
  C.update({i:dict()})

pbar = tqdm(caida_2007_G_test.nodes())
for y in pbar:

  for i in range(int(c*(eps**-2))): #Update the counters for each sample
    for ab in set_of_edges[i]:
      if nodes_connected(caida_2007_G_test, str(s(ab)[0]), y) and nodes_connected(caida_2007_G_test, str(s(ab)[1]), y):
        C[i][s(ab)] += 1
 
  for yx in list(caida_2007_G_test.edges(y)):
    yx = s(yx)
    # xy = tuple(reversed(yx))
    for i in range(int(c*(eps**-2))):
      if yx in set_of_edges[i]:
        current_maximum[i] = max(current_maximum[i], C[i][yx]/set_of_ervs[i][yx])
      else:
        R_yx = caida_2007_oracle[s(yx)] + beta if s(yx) in caida_2007_oracle else beta
        set_of_ervs[i][yx] = np.random.exponential(1)
        C[i][yx] = 0
        set_of_edges[i].add(yx)
        
        temp_score = R_yx / set_of_ervs[i][yx]
        set_of_edge_score[(yx[0],yx[1],i)] = temp_score

  M =  np.inf
  sum_of_s_i = 0
  set_of_edge_score_sorted = dict()
  set_of_edge_score_sorted = sort_dict_by_value(set_of_edge_score, True)
  for el  in set_of_edge_score_sorted:
    sum_of_s_i += 1
    if sum_of_s_i > H :
      break
    else:
      M = set_of_edge_score_sorted[el]
  copy = dict()
  copy = dict(set_of_edge_score)
  set_of_edge_score = dict()

  for e in copy:
    score = copy[e]
    edge = s(tuple((e[0],e[1])))
    idx = e[2]
    if score < M:
      set_of_edges[idx].remove(s(edge))
      del set_of_ervs[idx][s(edge)]
      del C[idx][s(edge)]
    else:
      set_of_edge_score[e] = score
          


for i in range(int(c*(eps**-2))):
  X.add(current_maximum[i])

temp_count= int(np.log(2) * sorted(X)[len(X)//2])
print("Predicted number of Triangles of caida_2007 #25: ", temp_count)
real_tri_count = sum(nx.triangles(caida_2007_G_test).values())/3
print("Real number of Triangles of caida_2007 #25: ", real_tri_count)

  0%|          | 0/25304 [00:00<?, ?it/s]

Predicted number of Triangles of caida_2007 #25:  39554
Real number of Triangles of caida_2007 #25:  38812.0
